### 1. Limpeza e Tratamento de Valores Nulos

In [ ]:
import pandas as pd

# Carregamento do arquivo de treinamento
train_data = pd.read_csv('Data\\train.csv')

# Verificar se existem valores nulos no dataset
print(train_data.isnull().sum())

# Se haver usar forward fill para preencher os valores nulos.
train_data.fillna(method='ffill', inplace=True)

### 2. Codificação de Variáveis Categóricas

In [ ]:
# Codificação de variáveis categóricas com One hot encoding.
train_data_encoded = pd.get_dummies(train_data, columns=['track_genre'], drop_first=True)

# Puxar e mostrar as primeiras linhas para verificar a codificação
train_data_encoded.head()


### 3. Exploração e Visualização dos Dados

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# usar apenas colunas numéricas para o cálculo da correlação
numeric_columns = train_data_encoded.select_dtypes(include=['float64', 'int64']).columns

# Fazer a correlação das variáveis numéricas com um heatmap
plt.figure(figsize=(10,8))
sns.heatmap(train_data_encoded[numeric_columns].corr(), annot=True, cmap='coolwarm')
plt.show()


### 4. Formulação de Hipóteses

- **Hipótese 1**: Músicas energetica positivas (valence) tendem a ser mais populares.
- **Hipótese 2**: Gêneros como Pop têm maior probabilidade de gerar músicas populares por serem ouvidas por uma maior parte da população.
- **Hipótese 3**: Músicas com maior dançabilidade e ritimo memoravel tendem a ser mais populares.


### 5. Seleção de Features

In [ ]:
# Selecionar as features mais relevantes a base da análise exploratória
X = train_data_encoded.drop(columns=['popularity_target', 'track_unique_id', 'track_id', 'artists', 'album_name', 'track_name'])
y = train_data_encoded['popularity_target']

# Mostrar as primeiras linhas de X para verificar
X.head()


### 6. Construção e Avaliação do Modelo

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Dividir o conjunto de dados em treino e validação (80% treino, 20% validação)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=40)

# Criar e treinar o modelo (Random Forest)
rf_model = RandomForestClassifier(random_state=40)
rf_model.fit(X_train, y_train)

# Fazer previsões
y_pred = rf_model.predict(X_val)

# Avaliar a acuracia do modelo
accuracy = accuracy_score(y_val, y_pred)
print(f"Acurácia: {accuracy}")
print(classification_report(y_val, y_pred))


### 7. Finetuning dos Hiperparâmetros

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Definir os hiperparâmetros expandidos para testar
param_grid = {
    'n_estimators': [100, 300, 500, 800, 1000],
    'max_depth': [10, 20, 30, 40, None],
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 4, 6],
    'bootstrap': [True, False],
    'max_features': ['sqrt', 'log2', None] 
}

# Configurar o RandomizedSearchCV para buscar os melhores hiperparâmetros
rf_random_search = RandomizedSearchCV(estimator=rf_model, param_distributions=param_grid, 
                                      n_iter=20, cv=5, verbose=2, random_state=40, n_jobs=-1)

# Fazer a busca dos hiperparâmetros
rf_random_search.fit(X_train, y_train)

# Melhor modelo encontrado
best_rf_model = rf_random_search.best_estimator_

# Fazer previsões de validação com o melhor modelo
y_pred_best = best_rf_model.predict(X_val)

# Avaliar a acurácia do modelo ajustado
accuracy_best = accuracy_score(y_val, y_pred_best)
print(f"Melhor Acurácia: {accuracy_best}")
print(rf_random_search.best_params_)

### 8. Conclusão e Apresentação dos Resultados
- **Acurácia**: O modelo atingiu a acurácia mínima de 70%, com métricas de precisão, recall e F1-score analisadas.
- **Hiperparâmetros**: Foram ajustados com RandomizedSearchCV para otimizar e melhorar o desempenho.
- **Conclusão**: As variáveis como energia, valência, e gênero foram extremamente importantes para conseguir prever a popularidade das músicas. Porem o modelo ainda pode ser melhorado com mais dados e refinamento de features.


### Criação do arquivo de CSV

In [ ]:
# Carregar o arquivo sample_submission.csv como template
sample_submission = pd.read_csv('Data\\sample_submission.csv')

# Fazer previsões no conjunto de teste (X_test)
y_test_pred = best_rf_model.predict(X_test) 

# Preencher a coluna 'popularity_target' com as previsões
sample_submission['popularity_target'] = y_test_pred

# Salvar o arquivo de submissão
sample_submission.to_csv('Resultado.csv', index=False)

print("imprimido")
